In [ ]:
# how to access the main colab drive to save libirary permentaluy
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

In [ ]:
!pip install --target=$nb_path git+https://github.com/boudinfl/pke.git
!pip install --target=$nb_path wikipedia
!pip install --target=$nb_path flashtext

In [ ]:
import nltk
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import pke
import wikipedia
from nltk.corpus import stopwords, words
WK_AI = wikipedia.summary("Artificialintelligence", sentences=10)
print(WK_AI)

Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals and humans. AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals.The term "artificial intelligence" had previously been used to describe machines that mimic and display "human" cognitive skills that are associated with the human mind, such as "learning" and "problem-solving". This definition has since been rejected by major AI researchers who now describe AI in terms of rationality and acting rationally, which does not limit how intelligence can be articulated.AI applications include advanced web search engines (e.g., Google), recommendation systems (used by YouTube, Amazon and Netflix), understanding human speech (such as Siri and Alexa), self-driving cars (e.g., Tesla), automated decision-making and competing at 

In [ ]:
def clean_text(f):
    
    lines = f
    all_words = lines.split()  # split all words, but it not words only its contain number and other things

    # step 1 : to make words contain alphabetic only
    alphabetic_only = [word for word in all_words if word.isalpha()]  # get  alphabetic only

    # step 2 : remove non-english word
    # words_nltk = set(words.words())  ## english word
    # english_alphabetic_only = [word for word in alphabetic_only if word in words_nltk]

    # step 3 : convert all words into lowercase
    lower_case_only = [word.lower() for word in alphabetic_only]

    # step 4 : drop all the stop words
    stopwords_nltk = set(stopwords.words('english'))  ## all stop words
    clean_words = [word for word in lower_case_only if word not in stopwords_nltk]
    return clean_words

In [ ]:
text = ' '.join(clean_text(WK_AI))
print(text)

artificial intelligence intelligence demonstrated opposed natural intelligence displayed animals ai research defined field study intelligent refers system perceives environment takes actions maximize chance achieving term previously used describe machines mimic display cognitive skills associated human definition since rejected major ai researchers describe ai terms rationality acting limit intelligence applications include advanced web search engines recommendation systems amazon understanding human speech siri cars automated competing highest level strategic game systems chess machines become increasingly tasks considered require often removed definition phenomenon known ai optical character recognition frequently excluded things considered become routine intelligence founded academic discipline years since experienced several waves followed disappointment loss funding followed new success renewed ai research tried discarded many different approaches since including simulating modeli

In [ ]:
def pke_modeling(model,text1):
  
  # initialize keyphrase extraction model, here TopicRank
  extractor = model

  # load the content of the document, here document is expected to be a simple 
  # test string and preprocessing is carried out using spacy
  extractor.load_document(input= text1 , language='en')

  # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
  # and adjectives (i.e. `(Noun|Adj)*`)
  extractor.candidate_selection()

  # candidate weighting, in the case of TopicRank: using a random walk algorithm
  extractor.candidate_weighting()

  # N-best selection, keyphrases contains the 10 highest scored candidates as+-+6
  # (keyphrase, score) tuples
  keyphrases = extractor.get_n_best(n=10)
  keys = list(zip(*keyphrases))[0]
  print(keys)

In [ ]:
all_models = [pke.unsupervised.TextRank(),pke.unsupervised.SingleRank()]
pke_modeling(all_models[1],text)

In [ ]:
# keyphrase sentance mapping 
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

# summarized_text is text after clean 
summarized_text = text
sentences = tokenize_sentences(summarized_text)
filtered_keys = keys
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
        
print (keyword_sentence_mapping)